In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import random
from pprint import pprint
import seaborn as sns
import pdb
sns.set_style(style="whitegrid")
from matplotlib import rcParams

%matplotlib inline
plt.style.use('fivethirtyeight')
rcParams['axes.labelsize'] = 14
rcParams['xtick.labelsize'] = 12
rcParams['ytick.labelsize'] = 12
rcParams['text.color'] = 'BLACK'
rcParams['figure.figsize'] = 16,8

In [5]:

# mean = [0, 0]
# cov = [[1, 0], [0, 1]]  # diagonal covariance
# Nobjs = 2000
# x, y = np.random.multivariate_normal(mean, cov, Nobjs).T
# #Add manual outlier
# x[0]=3.3
# y[0]=3.3
# X=np.array([x,y]).T
# X = pd.DataFrame(X,columns=['feat1','feat2'])
# plt.figure(figsize=(7,7))
# plt.plot(x,y,'bo');
# # plt.savefig('images/sample.png')
# plt.show()

# DATA

In [6]:
# Read data
df_11 = pd.read_csv('/home/ubuntu/Zabbix/Data/SRV_Data/SRV_Processed/SRV-FR-SQLN-11_Processed.csv')

# Missing Value Treatment
df_11 = df_11.set_index('Timestamp').fillna(df_11.mean())
print(df_11.isnull().sum().sum())

0


## Select Dimension

In [7]:
def select_feature(data): 
    '''
    Randomly select a feature of a dataframe
    '''
    return random.choice(data.columns)

## Select a value in dimension

In [8]:
def select_value(data,feat):
    '''
    Select values of 
    '''
    mini = data[feat].min()
    maxi = data[feat].max()
    return (maxi-mini)*np.random.random()+mini

In [12]:
var = select_feature(df_11) 
print(var)
select_value(df_11,var)

Free disk space on U:\LOGS


4256563200.0

## Split Data

In [13]:

def split_data(data, split_column, split_value):
    '''
    Split data based on the value of a column
    '''
    data_below = data[data[split_column] <= split_value]
    data_above = data[data[split_column] >  split_value]
    
    return data_below, data_above

In [27]:

var = select_feature(df_11) 
value = select_value(df_11,select_feature(df_11))
print(var,value)
a,b =split_data(df_11,
           var,
           value)

SQL Server, Statistics: Unsafe Auto-Params/sec 0.03246240852644829


In [30]:
print(value)

0.03246240852644829


In [31]:
print("a shape :",a.shape)
print("b shape :",b.shape)

a shape : (38432, 135)
b shape : (6838, 135)


In [32]:
def classify_data(data):
    
    label_column = data.values[:, -1]
    unique_classes, counts_unique_classes = np.unique(label_column, return_counts=True)

    index = counts_unique_classes.argmax()
    classification = unique_classes[index]
    
    return classification

classify_data(df_11)

87.17105445952873

## Isolation Tree

In [33]:
def isolation_tree(data,counter=0, max_depth=50):
    
    # End Loop
    if (counter == max_depth) or data.shape[0]<=1:
        classification = classify_data(data)
        return classification
    
    else:
        # Counter
        counter +=1
        
        # Select feature
        split_column = select_feature(data)
        
        # Select value
        split_value = select_value(data,split_column)

        # Split data
        data_below, data_above = split_data(data,split_column,split_value)
        
        # instantiate sub-tree      
        question = "{} <= {}".format(split_column, split_value)
        sub_tree = {question: []}
        
        # Recursive part
        below_answer = isolation_tree(data_below, counter,max_depth=max_depth)
        above_answer = isolation_tree(data_above, counter,max_depth=max_depth)
        
        if below_answer == above_answer:
            sub_tree = below_answer
        else:
            sub_tree[question].append(below_answer)
            sub_tree[question].append(above_answer)
        
        return sub_tree

In [35]:
tree = isolation_tree(df_11, max_depth=1)
print(tree)

{'SQL Server, Latches: Latch Waits/sec <= 9693.26553083682': [87.17105445952873, 77.9147]}


## Forest

In [38]:
def isolation_forest(df,n_trees=5, max_depth=5, subspace=256):
    forest = []

    for i in range(n_trees):
        # Sample the subspace
        if subspace<=1:
            df = df.sample(frac=subspace)
        else:
            df = df.sample(subspace)
        

        # Fit tree
        tree = isolation_tree(df,max_depth=max_depth)
        
        # Save tree to forest
        forest.append(tree)
    
    return forest

In [39]:
isolation_forest(df_11,n_trees=5,max_depth=1)

[{'Total disk space on Q: <= 254013760621.00363': [87.17105445952873, 84.493]},
 {'SQL Server, Statistics: Batch Requests/Sec <= 3370.3277430770495': [87.17105445952873,
   76.4382]},
 {'Free disk space on J: <= 17647665725.91596': [74.1268, 87.17105445952873]},
 {'SQL Server, Buffer Manager: Database pages <= 2621982.8812926398': [74.1268,
   87.17105445952873]},
 {'Free disk space on N: <= 39940172566.89792': [87.0226, 87.17105445952873]}]

In [40]:

def pathLength(example,iTree,path=0,trace=False):
    
    path=path+1
    question = list(iTree.keys())[0]
    
    feature_name, comparison_operator, value = question.split()
    
    
    # ask question
    if example[feature_name].values <= float(value):
        answer = iTree[question][0]
    else:
        answer = iTree[question][1]
        
        
    # base case
    if not isinstance(answer, dict):
        return path
    
    # recursive part
    else:
        residual_tree = answer
        return pathLength(example, residual_tree,path=path)

    return path

In [45]:
tree = isolation_tree(df_11,max_depth=100)

ValueError: attempt to get argmax of an empty sequence